<a href="https://colab.research.google.com/github/elbyvaz/data_engineering/blob/main/spark/titanic_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

# Spark entry point
spark = SparkSession \
    .builder \
    .appName("Titanic SQL") \
    .getOrCreate()

spark.version

In [ ]:
# Arquivo https://www.kaggle.com/c/titanic/data?select=train.csv
titanic_df = spark.read.csv('C:/Users/pedro.guerra/Downloads/AULAS/XPE/engenharia-dados/big-data-spark/aula2/titanic-3.csv',header='True',inferSchema='True')

titanic_df.printSchema()

In [ ]:
titanic_df.count()

In [ ]:
titanic_df.groupBy('survived').count().show()

In [ ]:
titanic_df.createOrReplaceTempView('table')

spark.sql("SELECT survived, count(*) FROM table GROUP BY survived").show()

In [ ]:
titanic_df.groupBy('survived').agg({"Fare": "avg"}).show()

In [ ]:
spark.sql("SELECT survived, avg(fare) FROM table GROUP BY survived").show()

In [ ]:
spark.sql("SELECT age, count(*) as age_count FROM table WHERE survived == 1 GROUP BY age ORDER BY age_count DESC").show()


In [ ]:
# UDFs ajudam plugar funções complexas no SQL.

def uppercase(str):
    return str.upper()

def ml_model(data):
    return model(data)

def eh_maior_idade(age):
    return age is None or age > 18


spark.udf.register("uppercase", uppercase)
spark.udf.register("eh_maior_idade", eh_maior_idade)

spark.sql("SELECT uppercase(Name), Name, age from table WHERE eh_maior_idade(Age) == True").show(5)


In [ ]:
# https://sparkbyexamples.com/spark/spark-write-dataframe-to-csv-file/

spark.sql("SELECT uppercase(Name), Age from table").write.option("header", True).format("csv").save("names2.csv")

In [ ]:
# https://sparkbyexamples.com/spark/spark-read-and-write-json-file/


spark.sql("SELECT age, count(*) as age_count FROM table WHERE survived == 1 GROUP BY age ORDER BY age_count DESC").write.format("json").save("ages2.json")

In [ ]:
# https://spark.apache.org/docs/latest/sql-data-sources-jdbc.html


# Saving data to a JDBC source
#jdbcDF.write \
#    .format("jdbc") \
#    .option("url", "jdbc:postgresql:dbserver") \
#    .option("dbtable", "schema.tablename") \
#    .option("user", "username") \
#    .option("password", "password") \
#    .save()
